## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto TASK 1

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np


## Scrape the wiki page

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
source.content
soup = BeautifulSoup(source.content, 'html.parser')

stat_table =soup.find_all('table', class_ = 'wikitable' )
stat_table = stat_table[0]

        
with open ('canada_postcode.txt', 'w') as r:
    for row in stat_table.find_all('tr'):

            
        for cell in row.find_all('td'):
           
            r.write('  ')
            r.write(cell.text)

     
            
headers = ['Postal Code', 'Borough', 'Neighborhood'] 

## Create a dataframe with the scraped data


In [3]:
import pandas as pd
df = pd.read_csv('canada_postcode.txt', sep= "  ", header = None, names = headers)

print(df.head(10))
df.shape

  Postal Code           Borough      Neighborhood
0         M1A      Not assigned      Not assigned
1         M2A      Not assigned      Not assigned
2         M3A        North York         Parkwoods
3         M4A        North York  Victoria Village
4         M5A  Downtown Toronto      Harbourfront
5         M5A  Downtown Toronto       Regent Park
6         M6A        North York  Lawrence Heights
7         M6A        North York    Lawrence Manor
8         M7A      Queen's Park      Not assigned
9         M8A      Not assigned      Not assigned


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


(289, 3)

## Drop cells with a borough that is Not assigned.


In [4]:
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
print(df.head())

  Postal Code           Borough      Neighborhood
0         M3A        North York         Parkwoods
1         M4A        North York  Victoria Village
2         M5A  Downtown Toronto      Harbourfront
3         M5A  Downtown Toronto       Regent Park
4         M6A        North York  Lawrence Heights


## Merge the nighborhoods with the same postal code.


In [5]:
df = (df.groupby(['Postal Code','Borough'],as_index = False, sort = False)['Neighborhood'].aggregate(', '.join))

df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## Copy the Borough  to the Not assigned neighborhoods.


In [6]:
df.Neighborhood = df.Borough.where(df.Neighborhood == 'Not assigned', df.Neighborhood)
df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [7]:
df.shape

(103, 3)

## Import the csv file with the coordinates data

In [8]:
path='https://cocl.us/Geospatial_data'
df2 = pd.read_csv(path)
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge the dataframes

In [10]:
df3 = pd.merge(df, df2, on='Postal Code')
df3.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
